In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
import os
import scipy as sp
from collections import defaultdict
from itertools import product
import gurobipy as gb
from gurobipy import GRB
from tqdm import tqdm
from itertools import chain
from itertools import permutations

repeats = json.load(open('../cleanerdata/repeats.json'))
repeats = {int(k):v for k,v in repeats.items()}

locdem = pd.read_excel('../cleanerdata/locdem.xlsx')

q = locdem['Number of pallets'].to_numpy().astype(float).tolist()
q = [0] + q
q.extend([v['dem'] for v in repeats.values()])

locs = pd.read_excel('../cleanerdata/locations.xlsx')
longs, lats = locs['long'].to_numpy().tolist(), locs['lat'].to_numpy().tolist()
longs.extend([longs[v['map']] for v in repeats.values()])
lats.extend([lats[v['map']] for v in repeats.values()])

distmat = pd.read_json('../cleanerdata/distmat.json').to_numpy()
timemat = pd.read_json('../cleanerdata/timemat.json').to_numpy()
def get(i):
    try:
        i = repeats[i]['map']
    except KeyError:
        pass
    return i

reverse_mapping = defaultdict(list)
for k,v in repeats.items():
    reverse_mapping[v['map']].append(k)
for k in reverse_mapping:
    reverse_mapping[k].append(k)

mappings = list(chain(*[permutations(ls,2) for ls in reverse_mapping.values()]))

def cost(i,j):
    if (i,j) in mappings:
        return 1e+8
    i,j = get(i), get(j)
    return distmat[i,j]

def time(i,j):
    i,j = get(i), get(j)
    return timemat[i,j]

def location(i):
    return longs[i], lats[i]

In [2]:
clusters = json.load(open('./clusters.json'))
list(map(lambda x: len(x), clusters))

[20, 20, 20, 20, 20, 20, 20, 20, 20]

In [3]:
Q = 9
iter_count = 0
primary_models = []
Ls = []

for cluster in clusters:
    Neighbours = list(sorted(cluster))
    all_nodes = [0] + Neighbours.copy()
    v = len(all_nodes)
    V = list(range(v))
    N = V.copy()
    N.remove(0)
    
    def Del(i):
        F = V.copy()
        F.remove(i)
        return F
    
    m = gb.Model()
    x = m.addVars(v,v, vtype=GRB.BINARY, name='x')
    m.setObjective(sum([cost(all_nodes[i], all_nodes[j]) * x[i,j] for i,j in product(V,V)]))
    for i in N:
        m.addConstr(sum([x[i,j] for j in Del(i)]) == 1)
    for j in N:
        m.addConstr(sum([x[i,j] for i in Del(j)]) == 1)
        
    l = m.addVar(lb=0.0, vtype=GRB.INTEGER, name='L')
    Ls.append(l)
    m.addConstr(sum([x[0,j] for j in Del(0)]) == l)
    
    u = m.addVars(V, vtype=GRB.CONTINUOUS, name='u')
    for i,j in filter(lambda tup: tup[0] != tup[1], product(N,N)):
        m.addConstr(u[i] - u[j] + Q*x[i,j] <= Q - q[j])
    for i in V:
        m.addConstr(u[i] >= q[i])
        
    for pair in mappings:
        if pair[0] in cluster and pair[1] in cluster:
            pair = Neighbours.index(pair[0])+1, Neighbours.index(pair[1])+1
            #print(Neighbours, pair)
            for length in tqdm(range(1,5)):
                all_groups = permutations(N,length)
                for group in all_groups:
                    if pair[0] in group or pair[1] in group:
                        continue
                    group = [pair[0]] + list(group) + [pair[1]]
                    edges = [(group[i], group[i+1]) for i in range(len(group) - 1)]
                    expr = 0
                    for edge in edges:
                        expr += x[edge]
                    m.addConstr(expr <= length)
                    
    m.Params.timelimit=600
    primary_models.append(m)

Set parameter Username
Academic license - for non-commercial use only - expires 2023-09-12


100%|█████████████████████████████████████████████| 4/4 [00:01<00:00,  3.04it/s]

Set parameter TimeLimit to value 600



100%|█████████████████████████████████████████████| 4/4 [00:01<00:00,  2.97it/s]

Set parameter TimeLimit to value 600


...

Set parameter TimeLimit to value 600



100%|█████████████████████████████████████████████| 4/4 [00:01<00:00,  3.14it/s]

Set parameter TimeLimit to value 600



100%|█████████████████████████████████████████████| 4/4 [00:01<00:00,  3.24it/s]

Set parameter TimeLimit to value 600


In [4]:
for model in tqdm(primary_models):
    model.optimize();

  0%|                                                     | 0/9 [00:00<?, ?it/s]

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[rosetta2])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 315082 rows, 463 columns and 1552934 nonzeros
Model fingerprint: 0x97ce7c24
Variable types: 21 continuous, 442 integer (441 binary)
Coefficient statistics:
  Matrix range     [1e+00, 9e+00]
  Objective range  [7e+01, 1e+08]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 8e+00]
Found heuristic solution: objective 1526791.0000
Presolve removed 8922 rows and 25 columns (presolve time = 5s) ...
Presolve removed 8922 rows and 25 columns
Presolve time: 5.29s
Presolved: 306160 rows, 438 columns, 1508650 nonzeros
Variable types: 20 continuous, 418 integer (418 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0      handle free variables                          7s

Starting sifting (using dual simplex for sub-problems)...

    Iter     Pivots    Primal Obj      Dual Ob

  1564   899 185897.330   18   57 433394.000 185897.330  57.1%  13.0  436s
  1580   914 185897.330   19   57 433394.000 185897.330  57.1%  13.2  440s
  1603   926 190476.474   21   52 433394.000 185897.330  57.1%  13.4  445s
  1641   941 221878.738   24   51 433394.000 185897.330  57.1%  13.5  452s
  1671   963 202306.488   26   46 433394.000 185897.330  57.1%  13.7  456s
  1702   977 206692.192   27   47 433394.000 185897.330  57.1%  13.7  461s
  1736   991 227007.877   30   49 433394.000 185897.330  57.1%  13.8  465s
  1770  1009 223551.144   33   54 433394.000 185897.330  57.1%  13.8  470s
  1787  1022 255226.391   33   46 433394.000 185897.330  57.1%  13.8  477s
  1810  1031 266245.145   34   51 433394.000 185897.330  57.1%  13.8  480s
  1850  1043 252805.169   37   45 433394.000 185897.330  57.1%  13.9  487s
  1872  1048 infeasible   38      433394.000 185897.330  57.1%  14.0  490s
  1925  1054 259228.270   42   47 433394.000 185897.330  57.1%  14.0  496s
  1970  1077 291309.323  

 11%|████▋                                     | 1/9 [10:00<1:20:02, 600.37s/it]

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[rosetta2])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 472402 rows, 463 columns and 2328410 nonzeros
Model fingerprint: 0x7c93cce2
Variable types: 21 continuous, 442 integer (441 binary)
Coefficient statistics:
  Matrix range     [1e+00, 9e+00]
  Objective range  [2e+01, 1e+08]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 8e+00]
Found heuristic solution: objective 2218477.0000
Presolve removed 28290 rows and 27 columns (presolve time = 5s) ...
Presolve removed 28290 rows and 27 columns
Presolve time: 7.90s
Presolved: 444112 rows, 436 columns, 2187944 nonzeros
Variable types: 20 continuous, 416 integer (416 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0   -0.0000000e+00   0.000000e+00   4.000000e+01     10s
     111    1.5604133e+05   0.000000e+00   0.000000e+00     10s
     111    1.5604133e+05   0.000000e+00

...

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[rosetta2])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 2202922 rows, 463 columns and 10858646 nonzeros
Model fingerprint: 0x4e01e2bd
Variable types: 21 continuous, 442 integer (441 binary)
Coefficient statistics:
  Matrix range     [1e+00, 9e+00]
  Objective range  [5e+02, 1e+08]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 8e+00]
Found heuristic solution: objective 1761041.0000
Presolve removed 22 rows and 23 columns (presolve time = 5s) ...
Presolve removed 609288 rows and 23 columns (presolve time = 10s) ...
Presolve removed 609288 rows and 23 columns (presolve time = 15s) ...
Presolve removed 609288 rows and 23 columns (presolve time = 20s) ...
Presolve removed 609288 rows and 23 columns (presolve time = 26s) ...
Presolve removed 1178570 rows and 51 columns (presolve time = 30s) ...
Presolve removed 1178570 rows and 51 columns (presolve time = 35s) ...
Presolv

   981   665 128695.490   20   26 418923.000 69642.0000  83.4%  17.1  600s

Cutting planes:
  Learned: 15
  Clique: 3
  MIR: 32
  Zero half: 3
  RLT: 4

Explored 1019 nodes (20193 simplex iterations) in 601.05 seconds (401.89 work units)
Thread count was 8 (of 8 available processors)

Solution count 10: 418923 419348 420658 ... 510494

Time limit reached
Best objective 4.189230000000e+05, best bound 6.964200000000e+04, gap 83.3759%


 89%|█████████████████████████████████████▎    | 8/9 [1:20:08<10:01, 601.39s/it]

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[rosetta2])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 4090762 rows, 463 columns and 20164358 nonzeros
Model fingerprint: 0x780cf8c3
Variable types: 21 continuous, 442 integer (441 binary)
Coefficient statistics:
  Matrix range     [1e+00, 9e+00]
  Objective range  [9e+01, 1e+08]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 8e+00]
Found heuristic solution: objective 1219448.0000
Presolve removed 22 rows and 23 columns (presolve time = 8s) ...
Presolve removed 22 rows and 23 columns (presolve time = 11s) ...
Presolve removed 22 rows and 23 columns (presolve time = 15s) ...
Presolve removed 1656962 rows and 23 columns (presolve time = 20s) ...
Presolve removed 1656962 rows and 23 columns (presolve time = 27s) ...
Presolve removed 1656962 rows and 23 columns (presolve time = 30s) ...
Presolve removed 1656962 rows and 23 columns (presolve time = 35s) ...
Presolve remo

100%|██████████████████████████████████████████| 9/9 [1:30:11<00:00, 601.33s/it]


In [5]:
model_solutions = {}
for i,model in enumerate(primary_models):
    soln = json.loads(model.getJSONSolution())
    model_solutions[i] = soln
json.dump(model_solutions, open('./primary_model_solutions.json', 'w'), indent=4)

In [6]:
s = 0
for model in primary_models:
    s += model.ObjVal
s

3951147.0

In [7]:
from itertools import permutations
K = [1,2,3]
print(*permutations(K,2))

(1, 2) (1, 3) (2, 1) (2, 3) (3, 1) (3, 2)
